In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import zipfile
from camaratools import *
%matplotlib inline
sns.set()

# Load and clean up data

This code uses ``camaratools`` to download the Brazilian legislature data, translate it, and clean it up. Please see the accompanying spreadsheet for metadata.

As a first step, we download the data. **We assume the data will be saved in ``../data/``**, i.e., one directory up in a folder called ``data``.

In [2]:
data_dir = '../data/'
downloadData(data_dir)

Done!


Next we translate the data and load it to one big dataframe. You can find an explanation for each feature in the excel spreadsheet in the metadata folder.

In [3]:
df = loadData(data_dir) # slow
df.head()

,name,id,docId,termStart,state,party,congressNum,categoryId,categoryTxt,subCategoryId,...,month,year,paymentNum,passengerName,trip,batch,numReimb,valueUnclear,congId,ideDocumento
0,ABELARDO CAMARINHA,141463,329,2011,SP,PSB,54,3,Fuel and lubricants,1,...,5,2012,0,NaN,NaN,612747,3973,0.0,1772,2319013
1,ABELARDO CAMARINHA,141463,329,2011,SP,PSB,54,3,Fuel and lubricants,1,...,5,2012,0,NaN,NaN,612747,3973,0.0,1772,2319015
2,ABELARDO CAMARINHA,141463,329,2011,SP,PSB,54,3,Fuel and lubricants,1,...,5,2012,0,NaN,NaN,612747,3973,0.0,1772,2319018
3,ABELARDO CAMARINHA,141463,329,2011,SP,PSB,54,3,Fuel and lubricants,1,...,10,2012,0,NaN,NaN,647402,4094,0.0,1772,2428891
4,ABELARDO CAMARINHA,141463,329,2011,SP,PSB,54,3,Fuel and lubricants,1,...,2,2012,0,NaN,NaN,593812,3890,0.0,1772,2257086


Next we clean up the Air Fare categroy a bit. There is **several** expenses that are reimbursments of air tickets (e.g., unused trips). This doesn't catch all instances (there are a lot of missed inputs in the data), but it manages to fix quite a few of them (around 80k entries).

In [4]:
df2 = cleanupAirFare(df)

Finally, we drop some of the features, and restrict our analysis to 1 term of congress starting in 2015 (period 2015-2017). The following are the parameters of the ``formatdf`` functions:

- ``start_date``: year for the analysis
- ``min_occur_name``: drop congresspeople with less than ``min_occur_name`` reimbursement requests
- ``min_occur_expenditures``: drop categories with less than ``min_occur_expenditures`` reimbursment requests (after cutting for year and congressmen)

In [5]:
df3 = formatdf(df2,start_date=2015,min_occur_name = 100, min_occur_expenditure = 500)
df3.head()

,name,id,termStart,state,party,congressNum,categoryId,categoryTxt,vendorId,vendorName,valueReimb,expenseDate,trip
1700648,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,05939467000115,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA,165.649994,2015-11-14 00:00:00,NaN
1700649,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,05939467000115,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA,59.480000,2015-12-10 00:00:00,NaN
1700650,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,02341470000144,ELETROBRAS DISTRIBUIÇÃO RORAIMA,130.949997,2015-11-27 00:00:00,NaN
1700651,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,02341470000144,ELETROBRAS DISTRIBUIÇÃO RORAIMA,193.059998,2015-12-30 00:00:00,NaN
1700652,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,00540252000103,PAPELARIA ABC Com. e Ind. LTDA.,310.250000,2015-02-23 00:00:00,NaN


We will save this formatted dataset in our data directory so we don't have to load it again.

In [6]:
file_name = 'congressBrazilData.csv'
df3.to_csv(data_dir+file_name,index=False)

You can load the dataframe in pandas using the following command:


In [7]:
df4 = pd.read_csv(data_dir+file_name)
df4.head()

,name,id,termStart,state,party,congressNum,categoryId,categoryTxt,vendorId,vendorName,valueReimb,expenseDate,trip
0,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,5.939467e+12,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA,165.65,2015-11-14 00:00:00,NaN
1,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,5.939467e+12,COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA,59.48,2015-12-10 00:00:00,NaN
2,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,2.341470e+12,ELETROBRAS DISTRIBUIÇÃO RORAIMA,130.95,2015-11-27 00:00:00,NaN
3,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,2.341470e+12,ELETROBRAS DISTRIBUIÇÃO RORAIMA,193.06,2015-12-30 00:00:00,NaN
4,ABEL MESQUITA JR.,178957,2015,RR,DEM,55,1,Maintenance of an office that supports parliam...,5.402520e+11,PAPELARIA ABC Com. e Ind. LTDA.,310.25,2015-02-23 00:00:00,NaN
